# 한글텍스트 분석

In [1]:
# import numpy as np
# import pandas as pd

In [2]:
#한국어 학습
from konlpy.tag import Kkma, Twitter, Komoran, Hannanum, Okt

In [3]:
komoran = Komoran()
okt = Okt()
kkma = Kkma()
han = Hannanum()

In [4]:
sentence = "아버지가방에들어가신다"

In [5]:
komoran.nouns(sentence)
okt.nouns(sentence)
kkma.nouns(sentence)
han.nouns(sentence)

['아버지가방에들어가']

In [6]:
komoran.pos(sentence)
okt.pos(sentence)
kkma.pos(sentence)
han.pos(sentence)

[('아버지가방에들어가', 'N'), ('이', 'J'), ('시ㄴ다', 'E')]

In [7]:
sentence = "2018[소방하반기대비]김세령국어최단기합격문풀(9-10월)"

In [8]:
komoran.nouns(sentence)
okt.nouns(sentence)
kkma.nouns(sentence)
han.nouns(sentence)

['2018[소방하반기대비]김세령국어최단기합격문풀', '9-10월']

In [9]:
komoran.pos(sentence)
okt.pos(sentence)
kkma.pos(sentence)
han.pos(sentence)

[('2018[소방하반기대비]김세령국어최단기합격문풀', 'N'), ('(', 'S'), ('9-10월', 'N'), (')', 'S')]

# 데이터 가져와서 하기

In [10]:
import numpy as np
import pandas as pd

data = pd.read_csv("kedi.csv",encoding='cp949')

#확인
data.head()
data.소분류코드.unique().size
type(data)

pandas.core.frame.DataFrame

In [11]:
#결과적으로 잘 나뉘는거 어느건지 확인
sentence = data.프로그램명[0] 

okt = Okt()

def make_nouns(x):
    return okt.nouns(x)

### 데이터전처리

In [12]:
data = data[['프로그램명','소분류코드','소분류']] #해당컬럼 가져오기
data.columns = ['program_name','code','code_name'] 
data = data.iloc[1:,:] #인덱싱 숫자를 1부터 시작
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True) #인덱싱 컬럼 없애기

In [13]:
X = data.program_name.apply(make_nouns) #시리즈
# (np.array(X[0][0])) #확인 #array('9', dtype='<U1')

In [14]:
#한글자짜리 없애기
def remove_one(x):
    new_x=[]
    for i in x:
        if len(i) > 1:
            new_x.append(i)
    return new_x

_X = X.apply(remove_one) #업데이트된X

In [15]:
full_text = [] #다 모인값
for i in X:
    full_text.extend(i)
full_text 

['명중',
 '명',
 '한글맞춤법',
 '명중',
 '명',
 '한글맞춤법',
 '참고도서',
 '전집',
 '제공',
 '명중',
 '명',
 '한글맞춤법',
 '참고도서',
 '제공',
 '스마트러닝',
 '명중',
 '명',
 '한글맞춤법',
 '스마트러닝',
 '명중',
 '명',
 '한글맞춤법',
 '스마트러닝',
 '언어',
 '이해',
 '기출',
 '문제',
 '해설',
 '특강',
 '완성',
 '학습',
 '국어',
 '한국어',
 '회화',
 '급',
 '정채',
 '필',
 '국어',
 '기본',
 '심화',
 '이론',
 '단',
 '급',
 '혜원',
 '혜원',
 '국어',
 '기본',
 '심화',
 '이론',
 '단',
 '급',
 '혜원',
 '혜원',
 '국어',
 '동형',
 '모의고사',
 '급',
 '혜원',
 '혜원',
 '국어',
 '이론',
 '입문',
 '급',
 '박병희',
 '양산',
 '국어',
 '한문',
 '특강',
 '급선',
 '국어',
 '단원',
 '기출',
 '족보',
 '정리',
 '특강',
 '급선',
 '국어',
 '최종',
 '정리',
 '찍기',
 '특강',
 '급유',
 '선선',
 '국어',
 '기본',
 '심화',
 '이론',
 '단',
 '급유',
 '선선',
 '국어',
 '약점',
 '체크',
 '모의고사',
 '급유',
 '선선',
 '국어문법',
 '개월',
 '완성',
 '특강',
 '급유',
 '선',
 '국어',
 '이론',
 '입문',
 '급',
 '혜원',
 '혜원',
 '국어',
 '한문',
 '특강',
 '지방직',
 '서울시',
 '대비',
 '약점',
 '체크',
 '모의고사',
 '국가',
 '직',
 '대비',
 '독해',
 '플러스',
 '특강',
 '배',
 '미진',
 '국어',
 '지방직',
 '서울시',
 '대비',
 '파이널',
 '실전',
 '모의고사',
 '학기',
 '한국어',
 '학당',
 '급',
 '학기',
 '한국어',
 '학당'

In [16]:
unique_word = list(set(full_text)) #리스트로 만들기
len(unique_word) #확인

16705

In [17]:
from collections import Counter #문자열이나, list 의 요소를 카운팅하여 많은 순으로 딕셔너리형태로 리턴

word_cnt = Counter(full_text)
common_word = word_cnt.most_common(1000)
unique_word = [ i for i,j in common_word]

## 정수인코딩 = 원핫인코딩

In [18]:
word_index = { k:v for k,v in enumerate(unique_word)}
index_word = { v:k for k,v in word_index.items()}

def label_encode(x): #분류가 정수로 안바껴서 바꿔줘야함
    encoded_x = []
    for i in x:
        encoded_x.append(index_word.get(i,0))
    return encoded_x

In [19]:
label_encoded = X.apply(label_encode)
label_encoded[0]

[0, 640, 0]

In [20]:
def vectorize_word(x,dimension=1000): #나올수 있는 최대 값으로 넣기
    t = np.zeros((x.shape[0],dimension))
    for k,v in enumerate(x.values):
        for i in v:
            if i < dimension:
                t[k,i] += 1
    return t

one_hot = vectorize_word(label_encoded)
one_hot #확인

array([[2., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.],
       ...,
       [3., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.]])

In [21]:
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

In [22]:
code_text = []
for i in data.code:
    code_text.append(i)
    
unique_code = list(set(code_text))
code_index = { k:v for k,v in enumerate(unique_code)} 
index_code = { v:k for k,v in code_index.items()}

In [23]:
data.code #를 집어넣어서 
def code_encode(x):
    encoded_x = []
    for i in x:
        encoded_x.append(index_code.get(i,0))
    return encoded_x
data.code.apply(code_encode) 

label_code = []
for i in data.code:
    label_code.append(index_code[i]) #index_code 각 번호일때 해당 정보 받아오려고 def
data['label'] = label_code

y = to_categorical(data.label.values) #카테고리컬 데이터 만들기

### "code_name","label" 추가컬럼가져오기

In [24]:
data.code_name
data[["code_name","label"]]

,code_name,label
0,한국어,59
1,한국어,59
2,한국어,59
3,한국어,59
4,한국어,59
...,...,...
125164,입시 기타,6
125165,입시 기타,6
125166,입시 기타,6
125167,입시 기타,6


In [25]:
# #### 나중에 활용때 추가 
# code_dictionary = {}
# for i in range(data.code_name.size):
#     code_dictionary[data.label[i]] = data.code_name[i]
# ####

In [26]:
dt = pd.DataFrame(np.c_[one_hot,y])

In [27]:
idx = np.random.choice(dt.index,len(dt.index),replace=False) #인덱스만큼 섞는다 #False::뽑은건 넣지 않는다

#확인
idx 
len(np.unique(idx))
idx[:]
int(0.8*len(idx)) 

100135

In [28]:
tr_idx = idx[:int(0.8*len(idx))]
te_idx = idx[int(0.8*len(idx)):]

X_train = dt.iloc[tr_idx,:1000].values
X_test = dt.iloc[te_idx,:1000].values
y_train = dt.iloc[tr_idx,1000:].values
y_test = dt.iloc[te_idx,1000:].values

In [29]:
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]

model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(input_shape,)))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(output_shape,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               256256    
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 121)               31097     
                                                                 
Total params: 550,265
Trainable params: 550,265
Non-trainable params: 0
_________________________________________________________________


In [30]:
optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer,
             loss=loss,
             metrics=metrics)

epochs = 10
batch_size = 300
validation_data = (X_test,y_test)
model.fit(X_train,
         y_train,
         epochs=epochs,
         batch_size=batch_size,
         validation_data=validation_data)

Epoch 1/10
334/334 [==============================] - 4s 11ms/step - loss: 2.0996 - accuracy: 0.5370 - val_loss: 1.3860 - val_accuracy: 0.6702
Epoch 2/10
334/334 [==============================] - 2s 7ms/step - loss: 1.2800 - accuracy: 0.6875 - val_loss: 1.2665 - val_accuracy: 0.6888
Epoch 3/10
334/334 [==============================] - 2s 7ms/step - loss: 1.1698 - accuracy: 0.7046 - val_loss: 1.2300 - val_accuracy: 0.6927
Epoch 4/10
334/334 [==============================] - 2s 7ms/step - loss: 1.1034 - accuracy: 0.7163 - val_loss: 1.2067 - val_accuracy: 0.6986
Epoch 5/10
334/334 [==============================] - 2s 7ms/step - loss: 1.0490 - accuracy: 0.7275 - val_loss: 1.1912 - val_accuracy: 0.7016
Epoch 6/10
334/334 [==============================] - 2s 7ms/step - loss: 1.0035 - accuracy: 0.7371 - val_loss: 1.1946 - val_accuracy: 0.7041
Epoch 7/10
334/334 [==============================] - 2s 7ms/step - loss: 0.9642 - accuracy: 0.7439 - val_loss: 1.1909 - val_accuracy: 0.7022
Epoch

In [31]:
###모델테스트
model.evaluate(X_test,y_test)

783/783 [==============================] - 1s 1ms/step - loss: 1.2245 - accuracy: 0.7016


[1.224493384361267, 0.7015658617019653]

In [32]:
#optimizer바꿔서 해봄
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]

model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(input_shape,)))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(output_shape,activation='softmax'))

optimizer = 'rmsprop'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer,
             loss=loss,
             metrics=metrics)

epochs = 10
batch_size = 300
validation_data = (X_test,y_test)
model.fit(X_train, 
          y_train, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_data=validation_data)

model.evaluate(X_test,y_test)

Epoch 1/10
334/334 [==============================] - 5s 13ms/step - loss: 2.0689 - accuracy: 0.5441 - val_loss: 1.4469 - val_accuracy: 0.6638
Epoch 2/10
334/334 [==============================] - 4s 11ms/step - loss: 1.3349 - accuracy: 0.6800 - val_loss: 1.3075 - val_accuracy: 0.6823
Epoch 3/10
334/334 [==============================] - 4s 11ms/step - loss: 1.2263 - accuracy: 0.6982 - val_loss: 1.2499 - val_accuracy: 0.6940
Epoch 4/10
334/334 [==============================] - 4s 11ms/step - loss: 1.1609 - accuracy: 0.7086 - val_loss: 1.2223 - val_accuracy: 0.6977
Epoch 5/10
334/334 [==============================] - 3s 9ms/step - loss: 1.1099 - accuracy: 0.7178 - val_loss: 1.1977 - val_accuracy: 0.7014
Epoch 6/10
334/334 [==============================] - 3s 8ms/step - loss: 1.0675 - accuracy: 0.7251 - val_loss: 1.1992 - val_accuracy: 0.7042
Epoch 7/10
334/334 [==============================] - 3s 8ms/step - loss: 1.0286 - accuracy: 0.7338 - val_loss: 1.1772 - val_accuracy: 0.7073
Ep

[1.1907882690429688, 0.7110329866409302]

## 활용

In [59]:
#X_train[0]의 모양대로 들어가야하고 출력은 57개의 출력값이 나옴
x = np.argmax(model.predict(X_train[[0]])) #argmax :: 최대값을 만들어주는 입력 x 

In [60]:
#### 나중에 활용때 추가 
code_dictionary = {}
for i in range(data.code_name.size):
    code_dictionary[data.label[i]] = data.code_name[i]
####

def vectorize_list(x,dimension=1000): #나올수 있는 최대 값
    t = np.zeros(dimension)
    for i in x:
        t[i] = 1
    return np.array(t).reshape(1,-1)

def predict_subject(subject):
    x = okt.nouns(subject)
    x = label_encode(x)
    x = vectorize_list(x)
    result = np.argmax(model.predict(x))
    result = code_dictionary[result]
    return f'입력한 {subject}의 분야는 {result}입니다.'

subject = okt.nouns(subject)
subject = label_encode(subject)
subject = '초등학교 논리수업'
predict_subject(subject)

'입력한 초등학교 논리수업의 분야는 영어입니다.'

In [61]:
subject = '이거를 라벨인코딩을 시켜줘야합니다'
predict_subject(subject)

'입력한 이거를 라벨인코딩을 시켜줘야합니다의 분야는 영어입니다.'